In [1]:
%load_ext autoreload
%autoreload

In [1]:
#from dipy.data import get_data
from nibabel import trackvis as tv
from dipy.segment.clustering import QuickBundles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

In [10]:
brn_avg = nib.load("./HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii")
print(type(brn_avg.header))
ba_data = brn_avg.get_data()
print(type(ba_data))
brn_avg_em = nib.load("./HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500_Eigenmaps.dtseries.nii")
print(type(brn_avg_em))


TypeError: 'Cifti2Header' object is not callable

In [4]:
import dipy.reconst.dti as dti

<function dipy.data.get_data>